In [1]:
##import shit
import random

from fastai import *
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re,string,unicodedata

#FastAI
import fastai
from fastai import *
from fastai.text import *
import torch
#Functional Tool
from functools import partial
#NLTK
import nltk
from nltk.tokenize.toktok import ToktokTokenizer

#SK Learn libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn import metrics
from fastai.vision.all import *
from IPython.display import Image
from torch.nn import CrossEntropyLoss

In [2]:
gendata_V1 = pd.read_csv("/Users/holee/Desktop/train.csv")

In [3]:
list_all = []
for i in gendata_V1["protein_sequence"]:
  list_all.append(" ".join(i))


In [4]:
total_join = "".join(list_all)#把 iterable 中的所有字符串元素用空字符串粘

In [5]:
unique_word = list(set(total_join))
aaa = []
for i in gendata_V1["protein_sequence"]:
     aaa.append(L([*i]))

In [10]:
gendata_V1["fenkai"] = aaa

In [11]:
def translator(x):
    temp_list = []
    for i in x:
        for j in range(len(unique_word)):
            if i == unique_word[j]:
                temp_list.append(j)
    return L(temp_list)

gendata_V1["fenkai_mum"] = gendata_V1["fenkai"].apply(translator)


In [12]:
len(gendata_V1["fenkai_mum"])

31390

In [13]:
new_data = gendata_V1["fenkai_mum"].iloc[0:30]
new_data

0                    [6, 6, 6, 6, 15, 6, 6, 6, 2, 6, 2, 2, 14, 12, 6, 1, 12, 16, 16, 7, 18, 20, 2, 1, 6, 14, 20, 9, 0, 1, 5, 9, 16, 5, 9, 2, 12, 9, 15, 14, 7, 14, 16, 2, 16, 14, 3, 18, 15, 7, 6, 14, 7, 7, 1, 7, 16, 8, 19, 14, 6, 12, 18, 0, 6, 5, 16, 9, 5, 6, 13, 12, 7, 9, 2, 12, 14, 14, 12, 14, 16, 14, 16, 16, 8, 15, 1, 14, 2, 14, 16, 1, 16, 14, 12, 1, 6, 18, 11, 1, ...]
1                  [6, 6, 6, 7, 14, 12, 1, 2, 19, 11, 12, 12, 12, 9, 6, 14, 6, 14, 0, 16, 14, 9, 13, 2, 1, 0, 12, 13, 12, 12, 0, 9, 8, 14, 13, 9, 1, 9, 9, 9, 9, 7, 2, 14, 13, 9, 2, 0, 6, 0, 9, 9, 6, 0, 9, 16, 6, 20, 12, 7, 14, 7, 12, 11, 16, 14, 0, 8, 16, 18, 1, 6, 0, 12, 12, 12, 14, 18, 12, 15, 1, 6, 12, 16, 19, 1, 8, 14, 15, 18, 14, 6, 15, 15, 2, 9, 15, 2, 12, 12, ...]
2                     [6, 6, 6, 5, 13, 8, 1, 9, 6, 8, 13, 17, 9, 18, 2, 13, 13, 6, 14, 13, 14, 13, 8, 9, 6, 7, 6, 1, 0, 16, 9, 9, 2, 19, 8, 8, 9, 7, 2, 2, 6, 15, 7, 17, 17, 6, 8, 2, 14, 16, 6, 15, 11, 6, 11, 14, 15, 7, 18, 15, 15, 6, 17, 17, 0, 2, 6, 1

In [16]:
idx = np.array(list((set(range(len(new_data))))))
np.random.shuffle(idx)
train_idx = idx[:int(len(new_data)*0.8)].tolist()
test_idx = idx[int(len(new_data)*0.8):].tolist()

In [18]:
split = [train_idx,test_idx]

In [19]:
import random
from torch import tensor
from torch.utils.data import Dataset

In [20]:
class data_add(nn.Module):##创建class只传入init里面变量data_add(dataset=gendata_V1,split=split,is_test=False)
    def __init__(self,dataset,split,long_sentence = 10,is_test = False):
        self.is_test = is_test
        self.dataset = dataset
        self.split = split
        self.long_sentence = long_sentence

    def __len__(self):
        if not self.is_test:
            return len(self.split[0])
        else:
            return len(self.split[1])

    def __getitem__(self, idx):
        if not self.is_test:
            idx_pool = self.split[0]
        else:
            idx_pool = self.split[1]

        df_idx = idx_pool[idx]  # 这里取正确索引
        full_sentence = self.dataset["fenkai_mum"].iloc[df_idx]

        if len(full_sentence) <= self.long_sentence + 1:
            templist = (self.long_sentence + 1)*[1000]
            templist[:self.long_sentence] = full_sentence
            x = templist[0:self.long_sentence]
            y = templist[1:self.long_sentence+1]
        else:
            x = full_sentence[0:self.long_sentence]
            y = full_sentence[1:self.long_sentence+1]

        new_y = 10 * [100]  # padding value 100 as in original
        # Copy y elements into new_y (truncate or pad)
        new_y[:len(y)] = y[:10]
        return tensor(x), tensor(new_y)




data_train_1 = data_add(dataset=gendata_V1,split=split,is_test=False)
data_test_1 = data_add(dataset=gendata_V1,split=split,is_test=True)



dl_train = DataLoader(data_train_1,batch_size=10)
dl_test = DataLoader(data_test_1,batch_size=10)
dls = DataLoaders(dl_train,dl_test)

In [21]:
dls.one_batch()

(tensor([[ 6,  6, 12, 11,  9,  1,  1, 18,  1, 14],
         [ 6,  6, 15, 13, 14,  7,  6, 12, 12,  6],
         [ 6,  6,  6, 13, 14,  2,  9,  8,  6, 18],
         [ 6,  6,  2,  6, 11, 13,  7, 18, 13, 16],
         [ 6,  6,  3,  8, 12,  1, 15, 13, 16, 10],
         [ 6,  6,  1,  7, 12, 18,  8,  8,  6, 20],
         [ 6,  6,  9,  9,  5, 13, 14,  1,  9, 11],
         [ 6,  6,  1,  7,  6,  1,  8,  5,  7, 13],
         [ 6,  6, 11, 19,  1, 11,  0,  7,  6,  8],
         [ 6,  6, 10,  5, 20,  9,  9,  8, 16, 18]]),
 tensor([[ 6, 12, 11,  9,  1,  1, 18,  1, 14, 13],
         [ 6, 15, 13, 14,  7,  6, 12, 12,  6,  9],
         [ 6,  6, 13, 14,  2,  9,  8,  6, 18,  1],
         [ 6,  2,  6, 11, 13,  7, 18, 13, 16,  6],
         [ 6,  3,  8, 12,  1, 15, 13, 16, 10, 16],
         [ 6,  1,  7, 12, 18,  8,  8,  6, 20,  1],
         [ 6,  9,  9,  5, 13, 14,  1,  9, 11,  0],
         [ 6,  1,  7,  6,  1,  8,  5,  7, 13,  2],
         [ 6, 11, 19,  1, 11,  0,  7,  6,  8,  3],
         [ 6, 10,  5, 20,  9,

In [ ]:
# model in PyTorch
# class LMModel1(Module):
#     def __init__(self, vocab_sz, n_hidden):
#         self.i_h = nn.Embedding(vocab_sz, n_hidden)  
#         self.h_h = nn.Linear(n_hidden, n_hidden)     
#         self.h_o = nn.Linear(n_hidden,vocab_sz)
        
#     def forward(self, x):
#         h = F.relu(self.h_h(self.i_h(x[:,0])))
#         h = h + self.i_h(x[:,1])
#         h = F.relu(self.h_h(h))
#         h = h + self.i_h(x[:,2])
#         h = F.relu(self.h_h(h))
#         return self.h_o(h)
    
# learn = Learner(dls, LMModel1(len(vocab), 64), loss_func=F.cross_entropy, metrics=accuracy)
# learn.fit_one_cycle(4, 1e-3)

In [22]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [23]:
class DropOutput(Callback):
    def after_pred(self): self.learn.pred = self. pred[0]

In [24]:
learn = Learner(dls,model,loss_func=CrossEntropyLossFlat(),cbs=[DropOutput], metrics=Perplexity())

In [ ]:
#learn = learn.to_fp16() if your divice is support cuda you can use it 
learn.fit(1)

epoch,train_loss,valid_loss,perplexity,time
0,4.575524,8.941432,7642.132812,00:16


## save the trained model

In [26]:
torch.save(learn, 'full_model.pth')
loaded_learn = torch.load('full_model.pth')

In [27]:
loaded_learn.model.eval()
x = tensor([16, 16,  4, 14,  0, 15,  1,  5, 13])
x = x.unsqueeze(0)
outputs = loaded_learn.model(x)
logits = outputs[0]
predicted_token_ids = logits.argmax(dim=-1)
predicted_token_ids

tensor([[6, 6, 6, 6, 6, 6, 6, 6, 6]])